# Polygon Stock Market API Tools

>[Polygon](https://polygon.io/) The Polygon.io Stocks API provides REST endpoints that let you query the latest market data from all US stock exchanges.

This notebook uses tools to get stock market data like the latest quote and news for a ticker from Polygon.

In [ ]:
import getpass
import os

os.environ["POLYGON_API_KEY"] = getpass.getpass()

In [2]:
from langchain_community.tools.polygon.last_quote import PolygonLastQuote
from langchain_community.tools.polygon.ticker_news import PolygonTickerNews
from langchain_community.utilities.polygon import PolygonAPIWrapper

In [8]:
api_wrapper = PolygonAPIWrapper()
ticker = "AAPL"

### Get latest quote for ticker

In [12]:
last_quote_tool = PolygonLastQuote(api_wrapper=api_wrapper)

In [13]:
last_quote_tool.run(ticker)

'{"P": 182.01, "S": 2, "T": "AAPL", "X": 11, "i": [604], "p": 182, "q": 3500343, "s": 7, "t": 1708439752930345226, "x": 12, "y": 1708439752930331680, "z": 3}'

### Get latest news for ticker

In [14]:
ticker_news_tool = PolygonTickerNews(api_wrapper=api_wrapper)

In [15]:
ticker_news_tool.run(ticker)

'[{"id": "5goYCKw3ZsHlJYwN1XX8AJCyybZuvfJeoUz38SzlYBM", "publisher": {"name": "The Motley Fool", "homepage_url": "https://www.fool.com/", "logo_url": "https://s3.polygon.io/public/assets/news/logos/themotleyfool.svg", "favicon_url": "https://s3.polygon.io/public/assets/news/favicons/themotleyfool.ico"}, "title": "Vanguard Total Stock Market ETF: Buy, Sell, or Hold?", "author": "newsfeedback@fool.com (Justin Pope)", "published_utc": "2024-02-20T12:30:00Z", "article_url": "https://www.fool.com/investing/2024/02/20/vanguard-total-stock-market-etf-buy-sell-or-hold/", "tickers": ["VTI", "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "TSLA", "GOOG"], "image_url": "https://g.foolcdn.com/editorial/images/765208/getty-buy-sell-hold-stocks-decide-ratings-analysts.jpg", "description": "This is the ultimate tool to diversify your portfolio.", "keywords": ["investing"]}, {"id": "F2nshszd6rBX-JJMHtbPuA325ZBizcq7iIJ7PBK_114", "publisher": {"name": "MarketWatch", "homepage_url": "https://www.marketw